In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, platform, warnings
from datasets import load_dataset
from trl import SFTTrainer

In [3]:
import glob
root_dataset=os.path.join("..")
dataset_dir=os.path.join(root_dataset,"dataset")
dataset_train_path=glob.glob(os.path.join(dataset_dir,"train","*.csv"))

In [4]:
torch.range(4,4).to("cuda")

/tmp/ipykernel_10769/2338294619.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(4,4).to("cuda")


tensor([4.], device='cuda:0')

In [5]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: A100-SXM-80GB
Compute Capability: (8, 0)
Total Memory: 85199093760 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 3.10.0-1160.el7.x86_64
Python Version: 3.10.12


In [6]:
# Pre trained model
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct" 

# # Dataset name
# dataset_name = "vicgalle/alpaca-gpt4"

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = "t1"

In [7]:
import json
import os
from pathlib import Path
law_dir=os.path.join(root_dataset,"wcx_ptt_export_280524")
law_list=glob.glob(os.path.join(law_dir,"*.json"))
law_data = {}
for i in law_list:
    key=Path(i).stem
    law_data[key]={}
    with open(i) as json_file:
        json_data = json.load(json_file)
    for j in json_data:
        law_data[key][j['section_num']]=j['section_content']

In [8]:
from datasets import Dataset,DatasetDict
import pandas as pd
df = pd.concat([pd.read_csv(i) for i in dataset_train_path])
ds = Dataset.from_pandas(df)
dataset=ds

In [9]:
df

,Unnamed: 0,idx,question,question_ori,answer,answer_ori,relevant_laws,irrelevant_laws,comment_1_1,comment_1_2
0,0,ห0015-1B-0002_2225,อะไรคือสิ่งที่ไม่ควรปิดบังเจ้าหน้าที่หรือสำนัก...,อะไรคือสิ่งที่ไม่ควรปิดบังเจ้าหน้าที่หรือสำนัก...,คำตอบ: สิ่งที่คนทั่วไปไม่ควรปิดบังคือข้อความจร...,คำตอบ: สิ่งที่คนทั่วไปไม่ควรปิดบังคือข้อความจร...,"[{'law_code': 'ห0015-1B-0002', 'sections': ['3...","[{'law_code': 'ส0065-1B-0001', 'sections': ['9...",NaN,NaN
1,1,ห0015-1B-0002_2235,การกระทำใดถือเป็นการแสดงข้อมูลที่ก่อให้เกิดควา...,การกระทำใดถือเป็นการแสดงข้อมูลที่ก่อให้เกิดควา...,คำตอบ: การนำส่งสำเนาหรือแสดงสมุดบัญชี เอกสาร ห...,คำตอบ: การนำส่งสำเนาหรือแสดงสมุดบัญชี เอกสาร ห...,"[{'law_code': 'ห0015-1B-0002', 'sections': ['3...","[{'law_code': 'ห0015-1B-0002', 'sections': ['8...",NaN,NaN
2,2,ห0015-1B-0002_2240,หากบริษัทหลักทรัพย์หลอกลวงประชาชนโดยการแสดงข้อ...,หากบริษัทหลักทรัพย์หลอกลวงประชาชนโดยการแสดงข้อ...,คำตอบ: หากบริษัทหลักทรัพย์หลอกลวงประชาชนโดยการ...,คำตอบ: หากบริษัทหลักทรัพย์หลอกลวงประชาชนโดยการ...,"[{'law_code': 'ห0015-1B-0002', 'sections': ['2...","[{'law_code': 'ก0173-1C-0001', 'sections': ['8...",NaN,NaN
3,3,ห0015-1B-0002_2247,ผู้ที่ได้รับมอบหมายให้จัดการทรัพย์สินของนิติบุ...,ผู้ที่ได้รับมอบหมายให้จัดการทรัพย์สินของนิติบุ...,คำตอบ: ผู้ที่ได้รับมอบหมายให้จัดการทรัพย์สินขอ...,คำตอบ: ผู้ที่ได้รับมอบหมายให้จัดการทรัพย์สินขอ...,"[{'law_code': 'ห0015-1B-0002', 'sections': ['3...","[{'law_code': 'ห0015-1B-0002', 'sections': ['3...",NaN,NaN
4,4,ห0015-1B-0002_2251,การครอบครองทรัพย์ของนิติบุคคลโดยบุคคลที่รับผิด...,การครอบครองทรัพย์ของนิติบุคคลโดยบุคคลที่รับผิด...,คำตอบ: การครอบครองทรัพย์ของนิติบุคคลโดยบุคคลที...,คำตอบ: การครอบครองทรัพย์ของนิติบุคคลโดยบุคคลที...,"[{'law_code': 'ห0015-1B-0002', 'sections': ['3...","[{'law_code': 'ห0015-1B-0002', 'sections': ['3...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3375,3375,ป0003-1D-0002_1043,การกระทำที่ไม่ผิดกฎหมาย แต่ขัดต่อศีลธรรมอันดีน...,การกระทำที่ไม่ผิดกฎหมาย แต่ขัดต่อศีลธรรมอันดีน...,คำตอบ:\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา ...,คำตอบ:\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา ...,"[{'law_code': 'ป0003-1D-0002', 'sections': ['1...","[{'law_code': 'ป0003-1D-0002', 'sections': ['1...",กฎหมายเก่าแล้ว ปัจจุบันคือมาตรา 150,NaN
3376,3376,ป0003-1D-0002_1022,การกระทำที่ไม่ชอบด้วยกฎหมาย หรือก่อขึ้นโดยไม่ส...,การกระทำที่ไม่ชอบด้วยกฎหมาย หรือก่อขึ้นโดยไม่ส...,คำตอบ:\n\nการกระทำที่ไม่ชอบด้วยกฎหมาย หรือเกิด...,คำตอบ:\n\nการกระทำที่ไม่ชอบด้วยกฎหมาย หรือเกิด...,"[{'law_code': 'ป0003-1D-0002', 'sections': ['1...","[{'law_code': 'ป0003-1D-0002', 'sections': ['1...",กฎหมายเก่าแล้วปัจจุบันคือมาตรา 149,NaN
3377,3377,ป0003-1D-0002_1399,ทำไมสัญญาที่มีเงื่อนไขขึ้นอยู่กับตัวผู้กู้ถึงถ...,ทำไมสัญญาที่มีเงื่อนไขขึ้นอยู่กับตัวผู้กู้ถึงถ...,คำตอบ:\n\nสัญญาที่มีเงื่อนไขขึ้นอยู่กับตัวผู้ก...,คำตอบ:\n\nสัญญาที่มีเงื่อนไขขึ้นอยู่กับตัวผู้ก...,"[{'law_code': 'ป0003-1D-0002', 'sections': ['1...","[{'law_code': 'ป0003-1D-0002', 'sections': ['1...",NaN,NaN
3378,3378,ป0003-1D-0002_1419,ใครสามารถสละประโยชน์จากเงื่อนไขเวลาได้,ใครสามารถสละประโยชน์จากเงื่อนไขเวลาได้,คำตอบ:\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา ...,คำตอบ:\n\nตามประมวลกฎหมายแพ่งและพาณิชย์ มาตรา ...,"[{'law_code': 'ป0003-1D-0002', 'sections': ['1...","[{'law_code': 'ป0003-1D-0002', 'sections': ['1...",NaN,NaN


In [10]:
# Load base model(llama-2-7b-hf) and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
dataset

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2', '__index_level_0__'],
    num_rows: 5772
})

In [12]:
import datetime
dataset.filter(lambda example: isinstance(eval(example['relevant_laws']),list))

Filter:   0%|          | 0/5772 [00:00<?, ? examples/s]

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2', '__index_level_0__'],
    num_rows: 5772
})

In [13]:
dataset[1000+1]

{'Unnamed: 0': 1001,
 'idx': 'ก0102-1B-0001_83',
 'question': 'หากกิจการต้องการเลิกกิจการ ต้องดำเนินการภายในระยะเวลาเท่าใด',
 'question_ori': 'หากกิจการต้องการเลิกกิจการ ต้องดำเนินการภายในระยะเวลาเท่าใด',
 'answer': 'คำตอบ: ตามกฎหมายการประกอบธุรกิจของคนต่างด้าว กิจการต้องแจ้งการเลิกกิจการต่อนายทะเบียนภายใน 15 วันนับแต่วันเลิกกิจการ\n\nคำอธิบายขยายความ:\nตามมาตรา 22 แห่งพระราชบัญญัติการประกอบธุรกิจของคนต่างด้าว พ.ศ. 2542 กำหนดให้ผู้รับใบอนุญาตหรือผู้รับหนังสือรับรองที่เลิกประกอบธุรกิจ ต้องแจ้งการเลิกกิจการต่อนายทะเบียนภายในระยะเวลา 15 วัน นับจากวันที่เลิกกิจการ โดยการแจ้งนั้นต้องเป็นไปตามแบบและวิธีการที่กำหนดในกฎกระทรวง ทั้งนี้ ข้อกำหนดนี้ใช้บังคับกับกรณีการย้ายสำนักงานหรือสถานที่ประกอบธุรกิจด้วยเช่นกัน',
 'answer_ori': 'คำตอบ: ตามกฎหมายการประกอบธุรกิจของคนต่างด้าว กิจการต้องแจ้งการเลิกกิจการต่อนายทะเบียนภายใน 15 วันนับแต่วันเลิกกิจการ\n\nคำอธิบายขยายความ:\nตามมาตรา 22 แห่งพระราชบัญญัติการประกอบธุรกิจของคนต่างด้าว พ.ศ. 2542 กำหนดให้ผู้รับใบอนุญาตหรือผู้รับหนังสือรับรองที่เลิกประกอบธุรกิ

In [14]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    outputs      = examples["answer"]
    list_law= [i for i in examples["relevant_laws"]]
    # list_txt_law=[]
    # for i in list_law:
    #     list_txt_law.append(law_data[i["law_code"][str(i['sections'][0])]])
    texts = []
    for i, output,law in zip(inputs, outputs,list_law):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        list_txt_law=[]
        for k in eval(law):
            if k["law_code"] not in set(law_data.keys()):
                pass
            else:
                if str(k['sections'][0]) not in set(law_data[k["law_code"]].keys()):
                    pass
                else:
                    for s in k['sections']:
                        if str(s) in set(law_data[k["law_code"]].keys()):
                            list_txt_law.append(law_data[k["law_code"]][str(s)].strip())
        if list_txt_law!=[]:
            source_information="- "+"\n- ".join(list_txt_law)
            source_information=source_information.strip()
            i=i.strip()
            content=f"""Query: {i}\nContinue to answer the query by using the Search Results:\n{source_information}"""
            text = tokenizer.apply_chat_template([{"role":"user","content":content},{"role":"assistant","content":output.strip()}], tokenize=False) + EOS_TOKEN
            texts.append(text)
        else:
            texts.append(None)
    return { "text" : texts, }

from datasets import load_dataset
dataset = dataset.map(formatting_prompts_func, batched = True,).filter(lambda example: isinstance(example['text'],str))

Map:   0%|          | 0/5772 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5772 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['Unnamed: 0', 'idx', 'question', 'question_ori', 'answer', 'answer_ori', 'relevant_laws', 'irrelevant_laws', 'comment_1_1', 'comment_1_2', '__index_level_0__', 'text'],
    num_rows: 4794
})

In [16]:
print(dataset["text"][2111])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Query: ใครเป็นผู้มีอำนาจในการแก้ไขวิธีการจัดการกองทุนรวม
Continue to answer the query by using the Search Results:
- พระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ พ.ศ. 2535 มาตรา 129/1 การแก้ไขเพิ่มเติมโครงการจัดการกองทุนรวมหรือแก้ไขวิธีการจัดการในกรณีดังต่อไปนี้บริษัทหลักทรัพย์อาจขอให้สำนักงานให้ความเห็นชอบแทนการขอมติของผู้ถือหน่วยลงทุนตามมาตรา 129 วรรคสอง ก็ได้
(1)การเปลี่ยนแปลงข้อกำหนดเกี่ยวกับการขายและการรับซื้อคืนหน่วยลงทุน
(2)การเพิ่มชนิดหน่วยลงทุนที่ไม่ทำให้สิทธิที่ผู้ถือหน่วยลงทุนมีอยู่เดิมด้อยลง
(3)การดำเนินการเพื่อให้สอดคล้องกับบทบัญญัติของพระราชบัญญัตินี้
(4)การแก้ไขเพิ่มเติมโครงการจัดการกองทุนรวมหรือแก้ไขวิธีการจัดการในลักษณะที่สำนักงานประกาศกำหนด
- พระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ พ.ศ. 2535 มาตรา 129 ในการจัดการกองทุนรวมบริษัทหลักทรัพย์ต้องดำเนินการตามโครงการจัดการกองทุนรวมที่ได้รับอนุมัติจากสำนักงาน
การแก้ไขเพิ่มเติมโครงการจัดการกองทุนรวมหรือแก้ไขวิธีการจัดการให้กระทำโดยการขอมติของผู้ถือหน่วยลงทุนซึ่งจะดำเนินกา

In [17]:
peft_config = LoraConfig(
    lora_alpha= 8,
    lora_dropout= 0.1,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [18]:
training_arguments = TrainingArguments(
    output_dir= "./results_mark3",
    num_train_epochs= 1,
    per_device_train_batch_size= 32,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 1000,
    logging_steps= 10,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "linear",
    report_to="wandb",
)

In [19]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
) 

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/4794 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
# Train model
trainer.train()

[2024-07-30 09:49:50,157] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wannaphong. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.133600
20,1.124500
30,0.845500
40,0.850500
50,0.755200
60,0.788600
70,0.720200


TrainOutput(global_step=75, training_loss=0.8797359275817871, metrics={'train_runtime': 8372.6992, 'train_samples_per_second': 0.573, 'train_steps_per_second': 0.009, 'total_flos': 1.456180926476452e+17, 'train_loss': 0.8797359275817871, 'epoch': 1.0})